In [62]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
import tensorflow as tf
from keras.metrics import Precision, Recall
from keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, Concatenate
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
from PIL import Image
import numpy as np
import os

def load_png_images_from_folder(folder_path, target_size=(28, 28)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            label = int(filename[0])  # Assuming the label is the first character in the filename
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            img = img.resize(target_size)  # Resize the image to a consistent size
            images.append(np.array(img))
            labels.append(label)
    return np.array(images), np.array(labels)

# Replace 'path/to/your/images' with the actual path to your folder containing .png files
image_folder_path = 'img/urdu_numerals/'
images, labels = load_png_images_from_folder(image_folder_path)

# Print the shape of the created dataset
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

In [ ]:
np.savez("datasets/urdu_numericals.npz", image=images, label = labels)

In [96]:
model =tf.keras.models.load_model(
        "models/numerical_model_convolutional.h5"
    ) 

model.summary()

Model: "mnist"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)               0     

In [ ]:
labels_x = to_categorical(labels)

In [97]:
model.summary()

Model: "mnist"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)               0     

In [123]:
new_model = Sequential()

for idx, layer in enumerate(model.layers):
    if layer.name not in ["dense_3","flatten_1","dense_2"]:
        new_model.add(layer)
    #new_model.add(MaxPooling2D(1, 1))
output_of_last_layer = new_model.layers[-1].output

#reshaped_output = Reshape((1, 1, 64))(output_of_last_layer)
#print(reshaped_output.shape)
# single_feature_normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=None)

dns = Conv2D(32, (1, 1), activation='relu', input_shape=(224,224))(output_of_last_layer)
dns_1 = MaxPooling2D(1, 1)(dns)
flat = Flatten()(dns_1)
# dns2 = Conv2D(32, (2, 2), activation='relu', input_shape=(224, 224))(new_model.layers[-1].output)
# dns_2 = MaxPooling2D(2, 2)(dns)
#merged = Concatenate(axis=-1)([dns, dns_1])

outputs = tf.keras.layers.Dense(10, activation="softmax")(flat)
print(output_of_last_layer.shape)
print(dns.shape)
print(dns_1.shape)
print(outputs.shape)

# input_layer_for_new_model = tf.keras.layers.Dense(
#     units=64, activation='softmax')(new_model.layers[-3].output)
print(new_model.layers[0].input.shape)
new_model = tf.keras.Model(inputs=new_model.layers[0].input, outputs=outputs)
#new_model.add(dns_1)
new_model.build(input_shape=(224, 224))
new_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

(None, 3, 3, 64)
(None, 3, 3, 32)
(None, 3, 3, 32)
(None, 10)
(None, 28, 28, 1)


ValueError: Found unexpected instance while processing input tensors for keras functional model. Expecting KerasTensor which is from tf.keras.Input() or output from keras layer call(). Got: 224

In [106]:
new_model.summary()

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3_input (InputLayer  [(None, 28, 28, 1)]       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                          

In [119]:
new_model.fit(images,labels_x, batch_size=32, epochs=10)

Epoch 1/10
3/3 [==============================] - 1s 18ms/step - loss: 3.1686 - accuracy: 0.2421
Epoch 2/10
3/3 [==============================] - 0s 17ms/step - loss: 1.4758 - accuracy: 0.5579
Epoch 3/10
3/3 [==============================] - 0s 18ms/step - loss: 0.9744 - accuracy: 0.7158
Epoch 4/10
3/3 [==============================] - 0s 19ms/step - loss: 0.8446 - accuracy: 0.7579
Epoch 5/10
3/3 [==============================] - 0s 18ms/step - loss: 0.8007 - accuracy: 0.8000
Epoch 6/10
3/3 [==============================] - 0s 19ms/step - loss: 0.7757 - accuracy: 0.8526
Epoch 7/10
3/3 [==============================] - 0s 18ms/step - loss: 0.7601 - accuracy: 0.8526
Epoch 8/10
3/3 [==============================] - 0s 18ms/step - loss: 0.7523 - accuracy: 0.8632
Epoch 9/10
3/3 [==============================] - 0s 19ms/step - loss: 0.7465 - accuracy: 0.8632
Epoch 10/10
3/3 [==============================] - 0s 18ms/step - loss: 0.7431 - accuracy: 0.8526


In [120]:
new_model.save("models/numerical_model_convolutional_2_fine_tuned_urdu.h5")

/Users/rusab1/Work/learntorch/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
